Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
!pip install icecream

In [2]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [25]:
import functools
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    helper.reset_calls = lambda: setattr(helper, 'calls', 0)
    return helper


@counter
def tsp_cost(tsp):
    #assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [26]:
from dataclasses import dataclass
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None


def fitness(individual):
    return-float(tsp_cost(individual.genome))


def parent_selection(population):
    #i1= np.random.randint(len(population))
    #return population[i1]
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def xover(p1: Individual, p2: Individual):
    genome = p1.genome.copy()
    i1= np.random.randint(len(genome))
    i2= np.random.randint(len(genome))
    while i1==i2:
        i2= np.random.randint(len(p1.genome))
    min1= min(i1,i2)
    max1= max(i1,i2)
    same= p1.genome[min1:max1+1]
    others= [item for item in p2.genome if item not in same]
    for i in range(len(genome)):
        if i < min1 or i > max1:
            genome[i]=others[0]
            others.pop(0)
    return Individual(genome)


def mutation(p: Individual):
    genome = p.genome.copy()
    x = 0
    while x < 0.1:
        index1 = np.random.randint(len(genome))
        index2 = np.random.randint(len(genome))
        while index1 == index2:
            index2 = np.random.randint(len(genome))
        #print("index1",index1)
        #print("index2",index2)
        min1 = min(index1,index2)
        max1 = max(index1,index2)
        #print("min1",min1)
        #print("max1",max1)
        #print("genome",genome)
        #print("genomesliced",genome[min1:max1+1])
        rev = list(reversed(genome[min1:max1+1]))
        #print("genome",genome)
        for i in range(len(genome)):
            if i >= min1 and i <= max1:
                #print("rev",rev)
                #print("index",i)
                #print("new_ele",rev[0])
                genome[i]=rev[0]
                #print("new",genome)
                rev.pop(0)
        x = np.random.random()
        #print("final",genome)
    return Individual(genome)

In [ ]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


In [28]:
import random
from tqdm import tqdm
POPULATION_SIZE = 10
#print(CITIES)
num=len(CITIES)
example = list(range(num))
#print(example)
population=[]
for i in range(POPULATION_SIZE):
    genome = random.sample(example,num)
    #print(genome)
    population.append(Individual(genome))
#print(population)
for i in population:
    i.fitness = fitness(i)

In [29]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .1:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [01:26<00:00, 116.10it/s]
ic| population[0].fitness: -387606.07862337795, tsp_cost.calls: 40000


In [30]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .9:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:30<00:00, 324.58it/s]
ic| population[0].fitness: -199162.5022580914, tsp_cost.calls: 40000


In [ ]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [32]:
import random
from tqdm import tqdm
POPULATION_SIZE = 10
#print(CITIES)
num=len(CITIES)
example = list(range(num))
#print(example)
population=[]
for i in range(POPULATION_SIZE):
    genome = random.sample(example,num)
    #print(genome)
    population.append(Individual(genome))
#print(population)
for i in population:
    i.fitness = fitness(i)

In [33]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .1:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:08<00:00, 1212.78it/s]
ic| population[0].fitness: -3986.759909214082, tsp_cost.calls: 40000


In [34]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .9:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:05<00:00, 1986.64it/s]
ic| population[0].fitness: -3793.3610811438284, tsp_cost.calls: 40000


In [ ]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


In [36]:
import random
from tqdm import tqdm
POPULATION_SIZE = 10
#print(CITIES)
num=len(CITIES)
example = list(range(num))
#print(example)
population=[]
for i in range(POPULATION_SIZE):
    genome = random.sample(example,num)
    #print(genome)
    population.append(Individual(genome))
#print(population)
for i in population:
    i.fitness = fitness(i)

In [37]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .1:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:14<00:00, 696.30it/s]
ic| population[0].fitness: -68986.31800235713, tsp_cost.calls: 40000


In [38]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .9:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:09<00:00, 1102.88it/s]
ic| population[0].fitness: -34783.45065467167, tsp_cost.calls: 40000


In [ ]:
CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676


In [40]:
import random
from tqdm import tqdm
POPULATION_SIZE = 10
#print(CITIES)
num=len(CITIES)
example = list(range(num))
#print(example)
population=[]
for i in range(POPULATION_SIZE):
    genome = random.sample(example,num)
    #print(genome)
    population.append(Individual(genome))
#print(population)
for i in population:
    i.fitness = fitness(i)

In [41]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .1:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:30<00:00, 324.40it/s]
ic| population[0].fitness: -141294.37459713023, tsp_cost.calls: 40000


In [42]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .9:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:13<00:00, 756.44it/s]
ic| population[0].fitness: -63014.98456678133, tsp_cost.calls: 40000


In [ ]:
CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


In [44]:
import random
from tqdm import tqdm
POPULATION_SIZE = 10
#print(CITIES)
num=len(CITIES)
example = list(range(num))
#print(example)
population=[]
for i in range(POPULATION_SIZE):
    genome = random.sample(example,num)
    #print(genome)
    population.append(Individual(genome))
#print(population)
for i in population:
    i.fitness = fitness(i)

In [45]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .1:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:06<00:00, 1550.08it/s]
ic| population[0].fitness: -822.5706248361558, tsp_cost.calls: 40000


In [46]:
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000
tsp_cost.reset_calls()
for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < .9:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = xover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

ic(population[0].fitness, tsp_cost.calls)
None

100%|██████████| 10000/10000 [00:04<00:00, 2307.49it/s]
ic| population[0].fitness: -822.5706248361558, tsp_cost.calls: 40000
